# Optymalizacja wielokryterialna

## Zespół:
- Szymon Szewczyk
- Łukasz Szyszka

In [1]:
import numpy as np
import timeit

In [21]:
def find_nondominated_points(X):
    # Lista punktów niezdominowanych
    P = []
    
    i = 0
    while i < len(X):
        Y = X[i]  # Wybierz aktywny punkt
        fl = 0  # Flaga zmian
        
        j = i + 1
        while j < len(X):
            if Y[0] <= X[j][0] and Y[1] <= X[j][1]:  # Jeśli Y dominuje X[j]
                X.pop(j)  # Usuń X[j]
            elif X[j][0] <= Y[0] and X[j][1] <= Y[1]:  # Jeśli X[j] dominuje Y
                X.pop(i)  # Usuń Y
                Y = X[j]  # Zaktualizuj Y
                fl = 1  # Zaznacz zmianę
                j = i  # Zresetuj pętlę dla nowego Y
            else:
                j += 1  # Przejdź do kolejnego elementu
        
        # Dodaj punkt Y do listy punktów niezdominowanych
        P.append(Y)
        
        # Usuń Y z listy przeglądanej, jeśli nie było żadnej zmiany
        if fl == 0:
            X.pop(i)
        else:
            i += 1  # Przejdź do następnego punktu

    return P

# Przykładowe użycie:
X = [(5, 5), (3, 6), (4, 4), (5, 3), (3, 3), (1, 8), (3, 4), (4, 5), (3, 10), (6, 6), (4, 1), (3, 5)]
P = find_nondominated_points(X)
print("Lista niezdominowanych punktów:", P)

Lista niezdominowanych punktów: [(1, 8), (3, 3), (4, 1)]


In [22]:
def find_nondominated_points_with_filter(X):
    # Lista punktów niezdominowanych
    P = []
    
    i = 0
    while i < len(X):
        Y = X[i]  # Wybierz aktywny punkt
        
        j = i + 1
        while j < len(X):
            if Y[0] <= X[j][0] and Y[1] <= X[j][1]:  # Jeśli Y dominuje X[j]
                X.pop(j)  # Usuń X[j]
            elif X[j][0] <= Y[0] and X[j][1] <= Y[1]:  # Jeśli X[j] dominuje Y
                X.pop(i)  # Usuń Y
                Y = X[j]  # Zaktualizuj Y
                j = i  # Zresetuj pętlę dla nowego Y
            else:
                j += 1  # Przejdź do kolejnego elementu
        
        # Dodaj punkt Y do listy punktów niezdominowanych
        P.append(Y)
        
        # Filtracja - usuń wszystkie elementy, które są zdominowane przez Y
        X = [x for x in X if not (Y[0] <= x[0] and Y[1] <= x[1])]
        
        # Usuń Y z przeglądanej listy X
        if Y in X:
            X.remove(Y)
        
        # Jeśli w liście X został jeden element, dodaj go do P i zakończ
        if len(X) == 1:
            P.append(X[0])
            break
        
        i = 0  # Resetujemy indeks po filtracji

    return P

# Przykładowe użycie:
X = [(5, 5), (3, 6), (4, 4), (5, 3), (3, 3), (1, 8), (3, 4), (4, 5), (3, 10), (6, 6), (4, 1), (3, 5)]
P = find_nondominated_points_with_filter(X)
print("Lista niezdominowanych punktów:", P)


Lista niezdominowanych punktów: [(1, 8), (3, 4), (4, 1)]


In [2]:
def naive_nondominated_no_filter(X):
    """
    Naive algorithm without filtering dominated points after finding each nondominated point.
    """
    P = []  # List of nondominated points

    i = 0
    while i < len(X):
        Y = X[i]
        fl = 0

        j = i + 1
        while j < len(X):
            if Y <= X[j]:
                X.pop(j)
            elif X[j] <= Y:
                X.pop(i)
                Y = X[j]
                fl = 1
            else:
                j += 1

        P.append(Y)
        if fl == 0:
            X.pop(i)
        else:
            i += 1

    return P

# Example usage
X1 = [(5,5), (3,6), (4,4), (5,3), (3,3), (1,8), (3,4), (4,5), (3,10), (6,6), (4, 1), (3, 5)]
P_no_filter = naive_nondominated_no_filter(X1[:])
print("Naive method (no filtration) nondominated points:", P_no_filter)


Naive method (no filtration) nondominated points: [(1, 8)]


In [4]:
def naive_nondominated_with_filtration(X):
    """
    Naive algorithm with filtration after finding each nondominated point.
    """
    P = []

    i = 0
    while len(X) > 1:
        Y = X[i]

        j = i + 1
        while j < len(X):
            if Y <= X[j]:
                X.pop(j)
            elif X[j] <= Y:
                Y = X.pop(j)
            else:
                j += 1

        P.append(Y)
        # Filtration: remove all points dominated by Y
        X = [x for x in X if not (Y <= x)]

    if len(X) == 1:
        P.append(X[0])

    return P

# Example usage
X2 = [(5,5), (3,6), (4,4), (5,3), (3,3), (1,8), (3,4), (4,5), (3,10), (6,6), (4, 1), (3, 5)]
P_with_filter = naive_nondominated_with_filtration(X2[:])
print("Naive method (with filtration) nondominated points:", P_with_filter)


Naive method (with filtration) nondominated points: [(1, 8)]


In [4]:
def ideal_point_method(X):
    """
    Ideal point method with filtration based on distances from the ideal point.
    """
    P = []  # List of nondominated points

    X = np.array(X)  # Convert to NumPy array for easier manipulation

    # Step 2: Calculate the ideal point as the minimum of each coordinate
    xmin = np.min(X, axis=0)

    while len(X) > 0:
        # Step 4: Calculate squared Euclidean distances from the ideal point
        distances = np.sum((X - xmin) ** 2, axis=1)

        # Step 5: Sort distances and get sorted indices
        sorted_indices = np.argsort(distances)

        # Step 8: Get the current point X(J(m)) based on sorted index
        current_point = X[sorted_indices[0]]

        # Add the current point to the list of nondominated points
        P.append(current_point.tolist())

        # Step 8: Remove all points dominated by the current point
        X = np.array([x for x in X if not np.all(current_point <= x)])

    return P

# Example usage
X3 = [(5,5), (3,6), (4,4), (5,3), (3,3), (1,8), (3,4), (4,5), (3,10), (6,6), (4, 1), (3, 5)]
P_ideal_point = ideal_point_method(X3[:])
print("Ideal point method nondominated points:", P_ideal_point)

Ideal point method nondominated points: [[3, 3], [4, 1], [1, 8]]


In [5]:
X = [(5,5), (3,6), (4,4), (5,3), (3,3), (1,8), (3,4), (4,5), (3,10), (6,6), (4, 1), (3, 5)]

# Measure execution time using timeit and display results

# Naive method without filtration
time_no_filter = timeit.timeit(lambda: naive_nondominated_no_filter(X[:]), number=1)
P_no_filter = naive_nondominated_no_filter(X[:])
print(f"Naive method (no filtration) took {time_no_filter:.6f} seconds")
print("Nondominated points (no filtration):", P_no_filter)

# Naive method with filtration
time_with_filter = timeit.timeit(lambda: naive_nondominated_with_filtration(X[:]), number=1)
P_with_filter = naive_nondominated_with_filtration(X[:])
print(f"Naive method (with filtration) took {time_with_filter:.6f} seconds")
print("Nondominated points (with filtration):", P_with_filter)

# Ideal point method
time_ideal_point = timeit.timeit(lambda: ideal_point_method(X[:]), number=1)
P_ideal_point = ideal_point_method(X[:])
print(f"Ideal point method took {time_ideal_point:.6f} seconds")
print("Nondominated points (ideal point method):", P_ideal_point)

Naive method (no filtration) took 0.000007 seconds
Nondominated points (no filtration): [(1, 8)]
Naive method (with filtration) took 0.000009 seconds
Nondominated points (with filtration): [(1, 8)]
Ideal point method took 0.000211 seconds
Nondominated points (ideal point method): [[3, 3], [4, 1], [1, 8]]
